imports

In [1]:
import torch
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from huggingface_hub import login

login(token="xxx")

In [3]:
pip show bitsandbytes torch


Name: bitsandbytes
Version: 0.49.1
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/bitsandbytes-foundation/bitsandbytes
Author: 
Author-email: Tim Dettmers <dettmers@cs.washington.edu>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: numpy, packaging, torch
Required-by: 
---
Name: torch
Version: 2.9.0+cu126
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org
Author: 
Author-email: PyTorch Team <packages@pytorch.org>
License: BSD-3-Clause
Location: /usr/local/lib/python3.12/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-cufile-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvshmem-cu12, nvidia-nvtx-cu12, setuptools, sy

loading llama model

In [5]:
%%capture
!pip uninstall -y unsloth unsloth_zoo
!pip install "unsloth[zoo]"

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 512
dtype = None
load_in_4bit = True
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-2-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

/tmp/ipython-input-2273464521.py:1: UserWarning: WARNING: Unsloth should be imported before [transformers, peft] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/3.87G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

defining LoRa args

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2026.1.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


preparing the dataset

In [5]:
dataset = load_dataset("cadaeic/2000-sample-synthetic-recipe-dataset", split="train")
print("Dataset size:", len(dataset))

def convert_to_chat(example):
    text = f"<s>[INST] {example['instruction']} [/INST]\n{example['output']}</s>"

    return {"text": text}

dataset = dataset.map(convert_to_chat, remove_columns=dataset.column_names)

README.md:   0%|          | 0.00/195 [00:00<?, ?B/s]

2000_samples_synthetic_dataset.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset size: 2000


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [26]:
from trl import SFTTrainer
from transformers import TrainingArguments
max_seq_length = 1024

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    warmup_steps=5,
    max_steps=70,
    learning_rate=2e-5,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    max_grad_norm=1.0,
),
)

In [27]:
trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1 | Total steps = 70
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 39,976,960 of 6,778,392,576 (0.59% trained)


Step,Training Loss
10,0.915700
20,1.021800
30,1.167900
40,1.103700
50,1.006800
60,1.152800
70,1.067100


wandb: WARNING URL not available in offline run


train/epoch,▁▂▃▄▆▇█▁▂▃▄▆▇█▁▂▃▄▆▇█▁▂▃▄▆▇██
train/global_step,▁▂▃▅▆▇█▁▂▃▅▆▇█▁▂▃▅▆▇█▁▂▃▅▆▇██
train/grad_norm,▅▇▃▃▁▁▁▅▁▁▂▁▁▁▅ ▃▅▃▄▃▅ █▄▅ ▅
train/learning_rate,█▇▆▅▄▃▂█▇▆▅▄▃▂█▇▆▅▄▃▂█▆▅▄▃▂▁
train/loss,█▇▆▄▅▄▄▃▃▃▃▃▃▂▂▄▄▂▂▂▃▁▃▅▄▃▅▄
total_flos,5812284165242880.0
train/epoch,0.28
train/global_step,70
train/grad_norm,0.63237
train/learning_rate,0.0
train/loss,1.0671


In [28]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-2-7b-bnb-4bit")


In [29]:
prompt = "[INST] How do I make chocolate cookies? [/INST]"

# Tokenize prompt
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate
output_ids = model.generate(
    **inputs,
    max_length=200,
    temperature=0.7,
    eos_token_id=tokenizer.eos_token_id
)
# Decode
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


[INST] How do I make chocolate cookies? [/INST]
 everybody will love this!



Ingredients:
1 c. sugar
1/2 c. shortening
1 egg
1 tsp. vanilla
1 tsp. baking soda
1 c. flour
1/2 c. cocoa
1/2 tsp. salt
1/2 c. chopped nuts

Directions:
1. Cream sugar, shortening and egg.
2. Add vanilla and soda.
3. Add flour, cocoa and salt.
4. Stir in nuts.
5. Drop by spoonfuls on greased cookie sheet.
6. Bake in a preheated 350° oven for 10 to 12 minutes.



In [30]:
prompt = "[INST] How do I make lasagna? [/INST]"

# Tokenize prompt
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate
output_ids = model.generate(
    **inputs,
    max_length=200,
    temperature=0.7,
    eos_token_id=tokenizer.eos_token_id
)
# Decode
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))


[INST] How do I make lasagna? [/INST]
 everyone, let me share this recipe with you.



Ingredients:
1 lb. lasagna noodles
1 lb. ground beef
1/2 c. chopped onion
1 can tomato sauce
1 can mushrooms
1/2 c. water
1/4 c. milk
1 c. grated cheese
1/2 c. Parmesan cheese
1/2 tsp. salt
1/4 tsp. pepper

Directions:
1. Cook ground beef until brown.
2. Add onion.
3. Add tomato sauce and mushrooms.
4. Add water and milk.
5. Add salt and pepper.
6. Add grated cheese and Parmesan cheese.
7. Mix well.

